In [ ]:
import time
import requests
import re
from pprint import pformat
from json import dumps
from kafka import KafkaProducer

In [ ]:
def filterData(item) :
    cleansed_data = {
        "id" : item['id'],
        "content" : item['content'],
        "sendTime" : item['sendTime'],
        "sendTimePersian" : item['sendTimePersian'],
        "senderName" : item['senderName'],
        "senderUsername" : item['senderUsername'],
        "type" : item['type'],
        "raw-data" : item
    } 
    return cleansed_data

In [ ]:
producer = KafkaProducer(bootstrap_servers=['kafka-broker:29092'], value_serializer=lambda x: dumps(x).encode('utf-8'),
                        key_serializer=str.encode )

In [ ]:
TOPIC_NAME="Sahamyab-Session_16"

In [ ]:
url = "https://www.sahamyab.com/guest/twiter/list?v=0.1"
delay = 10

cnt=0
while True:
    try : 
        response = requests.request('GET', url, headers={'User-Agent': 'Chrome/116'})
        if response.status_code == requests.codes.ok:
            items = response.json()['items']
            for item in items:
                cnt+=1
                print("-=-"*20)
                print(f"#{cnt:4} - {item['id']}")

                if not item.get('content') :
                    print("tweet is unacceptable")
                    continue
                else : 
                    data=filterData(item)
                    producer.send(TOPIC_NAME, value=data, key= item['id'])
        else:
            print("Error in fetch data: {err}".format(err=response.status_code))
        time.sleep(delay)
    except :
        continue